This is an educational implementation of z-codes.

Copyright, Prof. Dr. Jens Dittrich

https://bigdata.uni-saarland.de/

In [1]:
from operator import add
from textwrap import wrap

In [2]:
# convert OSM-coordinates into a z-code:
# notice that we use a string-representation of z-codes for educational purposes
# a real implementation would represent z-codes in a bit-vector or map them to signed integers
def tile2ZCode(zoom, x, y):
    if zoom == 0:
        return ''
    x_binary =  ((bin(x)[2:]).zfill(zoom))
    y_binary =  ((bin(y)[2:]).zfill(zoom))
    return ''.join(map(add, y_binary, x_binary))

tile2ZCode(3,2,1)

'000110'

In [3]:
# inverse of tile2ZCode
# convert Oz-code to SM-coordinates:
def zCode2Tile(zCode):
    assert( len(zCode)%2 ==0 )
    zoom = len(zCode)//2
    x = y = 0
    exponent = int(zoom)-1
    for i in range(0,len(zCode),2):
        y += int(zCode[i])*2**exponent
        x += int(zCode[i+1])*2**exponent
        exponent -= 1
    return (zoom, x, y)
    
zCode2Tile(tile2ZCode(2,3,1))

(2, 3, 1)

In [4]:
startEsc = '\033[1;'
middleEsc = 'm'
endEsc = '\033[1;m'

# color-codes a string in steps of <stepSize>
def colPrint(string, stepSize=2):
    color = 31
    for i in range(0,len(string),stepSize):
        prefix = string[i:i+stepSize]
        print(startEsc+str(color)+middleEsc+prefix+endEsc, end='')
        color += 1
colPrint('hallo welt')

hallo welt

In [5]:
# print all z-codes of a given zoom-level:
def printZCodes(zoom):
    for y in range(2 ** zoom):
        for x in range(2 ** zoom):
            zCode= tile2ZCode(zoom, x, y)
            colPrint(zCode)
            print(' ', end='')
        print()

printZCodes(3)

000000 000001 000100 000101 010000 010001 010100 010101 
000010 000011 000110 000111 010010 010011 010110 010111 
001000 001001 001100 001101 011000 011001 011100 011101 
001010 001011 001110 001111 011010 011011 011110 011111 
100000 100001 100100 100101 110000 110001 110100 110101 
100010 100011 100110 100111 110010 110011 110110 110111 
101000 101001 101100 101101 111000 111001 111100 111101 
101010 101011 101110 101111 111010 111011 111110 111111 


In [6]:
def computeZCodes(zoom):
    results = []
    for y in range(2 ** zoom):
        for x in range(2 ** zoom):
            zCode= tile2ZCode(zoom, x, y)
            results.append( (zCode,x,y) )
    return results

In [7]:
from ipycanvas import Canvas, hold_canvas

# visualize the data space in z-code order, i.e. draw the z-curve:
def visualizeZCodes(zoom):
    results = computeZCodes(zoom)
    results.sort()
    x_width, y_width = 300, 300
    canvas = Canvas(size=(x_width, y_width),sync_image_data=True)
    canvas.line_width = 1
    max_x = max_y = 2**zoom
    mult_x, mult_y  = x_width/max_x, y_width/max_y
    start = results.pop(0)
    start_x, start_y = start[1], start[2]
    with hold_canvas(canvas):
        while len(results)>0:
            next = results.pop(0)
            next_x, next_y = next[1], next[2]
            canvas.begin_path()
            canvas.move_to( (start_x+0.5)*mult_x, (start_y+0.5)*mult_y)
            canvas.line_to( (next_x+0.5)*mult_x, (next_y+0.5)*mult_y)
            canvas.stroke()
            start_x,start_y = next_x, next_y 
    return canvas

for zoom in range(1,6):
    display(visualizeZCodes(zoom))

Canvas(layout=Layout(height='300px', width='300px'), size=(300, 300), sync_image_data=True)

Canvas(layout=Layout(height='300px', width='300px'), size=(300, 300), sync_image_data=True)

Canvas(layout=Layout(height='300px', width='300px'), size=(300, 300), sync_image_data=True)

Canvas(layout=Layout(height='300px', width='300px'), size=(300, 300), sync_image_data=True)

Canvas(layout=Layout(height='300px', width='300px'), size=(300, 300), sync_image_data=True)